# Data Wrangling on WeRateDogs



## Table of Contents
- [Introduction](#intro)
- [Gather](#gather)
- [Assess](#assess)
- [Clean](#clean)

<a id='intro'></a>
## Introduction

The main purpose of this project is to put into practice key concepts learnt during this module. Tasks that will be carrier out on the following notebook are gathering, assessing and cleaning data that has been provided to us using different means. In order to do so Python 3 will be used. Also the solution provided will lean on the following set of python libraries.

In [141]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import tweepy
import json
from timeit import default_timer as timer
%matplotlib inline

<a id='gather'></a>
## Gather    

The first step in data analysis is data gathering. In this case data gathering will be comprised of the following steps:   
**1.-** Import the tweeter archive for WeRateDogs into the workspace   
**2.-** Download image predictions data from a given URL   
**3.-** Gather data from the Tweeter API to collect more relevant information   

#### 1.- Import the tweeter archive
This file has already been provided as a manual download. As such, it's available locally and just need to import it using pandas read_csv method for reading flat files into a pandas dataframe. Then, output the first row to verify that it has loaded successfully.

In [142]:
archive_df = pd.read_csv("twitter_archive_enhanced.csv")
archive_df.head(1)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None


#### 2.- Download image predictions data from a given URL   
Unlike the previous file, in this case the file's URL was given. Hence, we download the file programatically as this is less prone to errors and eases reproducibility.    
First download the file using python library **_requests_** and save the information downloaded in a file. Then, import the file similarly to how it was done in step \#1. Also output first row to verify everything has gone correctly. 

In [143]:
url = "https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv"
image_predictions = requests.get(url)
file_name = url.split('/')[-1]
with open(file_name, mode='w', encoding =image_predictions.encoding) as f:
    f.write(image_predictions.text)
image_df = pd.read_csv(file_name,sep='\t')
image_df.head(1)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True


#### 3.- Gather data from Tweeter by accessing their API 
The third step for this data gathering effort is to obtain various pieces of data from Tweeter directly by using their API. Please note that keys and tokens are stored in a separate file that is stored locally and is not part of source control. As such, these fields will need to be edited if this notebook was to used different tweeter app credentials.    
The idea is to query Tweeter's API through python's api library for tweeter (Tweepy). The idea is to get some more information for each of the tweets we have in df_archive dataframe. Then store json content into a txt file. Once the json text file is available parse it and create a new dataframe to work under python environment. 
So let's first define the function that will be used to parse json text file into a dataframe

In [144]:
def parseJson():
    jsonlist = []
    with open('tweet_json.txt','r',encoding='utf-8') as fjsonRead:
        for line in fjsonRead:
            json_dict = json.loads(line)
            tweet_id = json_dict['id']
            retw_count = json_dict['retweet_count']
            fav_count = json_dict['favorite_count']
            jsonlist.append({'tweet_id':tweet_id, 'retw_count':retw_count, 'fav_count':fav_count})
    return pd.DataFrame(jsonlist,columns=['tweet_id', 'retw_count', 'fav_count'])

Downloading information from the API for so many tweets it's a time consuming task. Thus, we would not like to do it every time this notebook is run since this operation can take 30 minutes on its own. Therefore if the file is present, it's assumed that the API has already been queried and data stored in the file. If this was the case, then proceed with parsing the file into a dataframe. Otherwise, query the API and store data in a text file before parsing.

In [145]:
try:
    tweeter_df = parseJson()
except FileNotFoundError as e:
    start = timer()
    with open('tweet_json.txt','w',encoding='utf-8') as f:
        with open('tweetIdsFailed.txt','w',encoding='utf-8') as ffail:
            with open('tweeterKeys.txt','r',encoding='utf-8') as f:
                getData = lambda line: line.split(' ')[0]
                consumer_key = getData(f.readline())
                consumer_secret = getData(f.readline())
                access_token = getData(f.readline())
                access_secret = getData(f.readline())
                auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
                auth.set_access_token(access_token, access_secret)
                api = tweepy.API(auth,wait_on_rate_limit=True,wait_on_rate_limit_notify=True)
                for row in archive_df['tweet_id']:
                    try:
                        tweet = api.get_status(row,tweet_mode='extended')
                        json.dump(tweet._json, f)
                        f.write('\n')
                    except tweepy.TweepError as e:
                        ffail.write(str(row)+'\n')
    end = timer()
    print('Time: {}'.format(end-start))
    tweeter_df = parseJson()
tweeter_df.head(1)

,tweet_id,retw_count,fav_count
0,892420643555336193,8366,38195


<a id='assess'></a>
## Assess

Once all data has been collected, we need to inspect each dataframe to find flaws in either contents or structure. Let's start with archive_df. Let's first have a general look by printing the info

In [147]:
archive_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

The information above is filtered data from the 5000+ Tweeter account. It's filtered taking into account only those tweets that contain dog ratings. However as it can be seen above some of the tweets are **re**tweets and so we can regard these as duplicated information. We could verify that **retweeted_status_id** == tweet_id for other row in the dataframe. The problem here is that the type for column retweeted_status_id is float64. For example,

In [146]:
print (archive_df[archive_df['retweeted_status_id'].notnull()].retweeted_status_id.values[0],
       archive_df[archive_df['retweeted_status_id'].notnull()].retweeted_status_id.astype('int64').values[0])

8.874739571039519e+17 887473957103951872


To me the above results are wrong. The number on the left, in floating point arithmetic, contains only 15 decimal digits. The exponent however indicates 17. Therefore when casting to an **int64** type, somehow the last two digits are made up...
# ASK ABOUT THIS BEHAVIOR
Therefore and unless this data is gathered it's impossible to verify whether all retweets are already contained in the archive_df in the form of normal tweets. At this point there are two options. 
> First, search for this missing information in the downloaded json file
> Or second, assumme that all retweets' information is already present in archive_df in the form of normal tweets

Let's take the first option. This also demonstrates the spirit of iteration for which there was much emphasis in the class. Let's first create a new column called **retweeted_status_id_2** that will contain the integer retweeted_id. This column will be in string format and so no information will be lost. Then, we will compare both retweeted_status_id with retweeted_status_id2 to verify that they are not the same. Let's _gather_ the data first

In [168]:
# Get tweet id for which the tweet was a retweet
retw = archive_df[archive_df['retweeted_status_id'].notnull()].tweet_id.values
archive_df['retweeted_status_id_2'] = np.NaN
with open('tweet_json.txt','r',encoding='utf-8') as fjsonRead:
    for line in fjsonRead:
        json_dict = json.loads(line)
        if json_dict['id'] in retw:
            archive_df.loc[archive_df[archive_df['tweet_id']==json_dict['id']].index,'retweeted_status_id_2'] = str(json_dict['retweeted_status']['id'])
        else:
            archive_df.loc[archive_df[archive_df['tweet_id']==json_dict['id']].index,'retweeted_status_id_2'] = np.NaN

Now that the information has been collected, let's compare both columns

In [170]:
retw = archive_df[archive_df['retweeted_status_id_2'].notnull()]
(retw.retweeted_status_id.astype('int64') - retw.retweeted_status_id_2.astype('int64')).values

array([ 0,  0,  0,  0,  0, -1, -3,  0,  0,  0,  0,  0,  0, -1,  0,  0,  0,
       -2,  0,  0,  0,  0,  0, -2, -1,  0, -1,  0, -2, -1, -1, -2,  0,  0,
        0, -2, -3, -2,  0, -2,  0,  0, -1, -1,  0,  0, -5, -1, -2, -1,  0,
        0,  0,  0,  0, -4, -5,  0,  0,  0,  0,  0,  0,  0, -2,  0,  0, -4,
        0,  0,  0, -2,  0,  0, -1, -2,  0,  0,  0,  0,  0,  0,  0,  0, -2,
        0, -1,  0,  0,  0,  0,  0, -5,  0,  0,  0,  0,  0,  0, -4,  0,  0,
        0,  0,  0, -1,  0,  0,  0,  0, -1,  0,  0,  0, -1,  0,  0, -5,  0,
        0, -1,  0,  0,  0,  0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  0,  0,
       -1,  0, -2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0,  0, -2,
        0,  0,  0, -1,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0, -1],
      dtype=int64)

We can see the results above. They indicate that the casting from float to int64 does not yield correct results and so it would have been wrong to analyse using a casting method. Therefore the information needs to be extracted from the API itself. Now that we have the right information let's see how many of these retweets are already present in the whole dataframe

In [173]:
sum(np.isin(retw.retweeted_status_id_2.astype('int64').values,archive_df.tweet_id.values))

145

It can be seen by the number 145 above that some entries in the archive_df are duplicates of original tweets.    
> **Some retweets contain duplicate information**     

Also mentioned above, some column types are wrong. Columns that are id(s) should not be float64. Also timestamp columns should be casted to datetime
> **Some column types are wrong**

In a similar way, when this twitter account replied to some tweets it's possible that both original and reply contain a score. In this case the second one overrides the first and so we should only consider the last score to be the valid one. Let's proceed as we did for the retweets

In [227]:
# Get tweet id for which the tweet was a retweet
retw = archive_df[archive_df['in_reply_to_status_id'].notnull()].tweet_id.values
archive_df['in_reply_to_status_id_2'] = np.NaN
with open('tweet_json.txt','r',encoding='utf-8') as fjsonRead:
    for line in fjsonRead:
        json_dict = json.loads(line)
        if json_dict['id'] in retw:
            if str(json_dict['in_reply_to_status_id']) == 'None':
                archive_df.loc[archive_df[archive_df['tweet_id']==json_dict['id']].index,'in_reply_to_status_id'] = np.NaN
                archive_df.loc[archive_df[archive_df['tweet_id']==json_dict['id']].index,'in_reply_to_user_id'] = np.NaN
            else:
                archive_df.loc[archive_df[archive_df['tweet_id']==json_dict['id']].index,'in_reply_to_status_id_2'] = str(json_dict['in_reply_to_status_id'])
        else:
            archive_df.loc[archive_df[archive_df['tweet_id']==json_dict['id']].index,'in_reply_to_status_id_2'] = np.NaN  
retw = archive_df[archive_df['in_reply_to_status_id_2'].notnull()]
print('Original message:')
print('    {}'.format( archive_df.loc[archive_df.tweet_id == np.int64(retw.iloc[5].in_reply_to_status_id_2)].text.values[0]))
print('In reply to the previous')
print('    {}'.format(retw.iloc[5].text))
numberreplies = sum(np.isin(retw.in_reply_to_status_id_2.astype('int64').values,archive_df.tweet_id.values))
print('Total number of replies for which the original is in archive_df : {}'.format(numberreplies))

Original message:
    This is Pipsy. He is a fluffball. Enjoys traveling the sea &amp; getting tangled in leash. 12/10 I would kill for Pipsy https://t.co/h9R0EwKd9X
In reply to the previous
    Ladies and gentlemen... I found Pipsy. He may have changed his name to Pablo, but he never changed his love for the sea. Pupgraded to 14/10 https://t.co/lVU5GyNFen
Total number of replies for which the original is in archive_df : 44


It can be seen from the above that there are 44 replies that need to be removed
> **Some replies contain duplicated score**

Coming back to archive_df.info(), it can be observed that some of the values in expanded_url are missing. This information needs to be gathered
> **Expanded_url column contains missing information**

Let's now check what happens with dog names. I guess that a regex statement has been used to extract dog names on this column. It's possible that the standard regex may extract wrong information, so let's find out.

In [248]:
print(archive_df.name.value_counts().index[0:40])

Index(['None', 'a', 'Charlie', 'Cooper', 'Lucy', 'Oliver', 'Lola', 'Tucker',
       'Penny', 'Winston', 'Bo', 'the', 'Sadie', 'an', 'Toby', 'Buddy',
       'Bailey', 'Daisy', 'Jax', 'Stanley', 'Leo', 'Dave', 'Jack', 'Rusty',
       'Milo', 'Oscar', 'Koda', 'Scout', 'Bella', 'Alfie', 'Bentley', 'Gus',
       'Larry', 'Sunny', 'George', 'Phil', 'Chester', 'very', 'Sammy',
       'Louis'],
      dtype='object')


It can be observed above that **a,** **an**, **very**, **the** ... are not real dog names. Also there is a huge number of missing names. From what I can see above, it seems that most errors share a similar pattern, that is the first letter is in lowercase, as opposed to proper names.
Let's develop this idea a bit further


In [249]:
names = pd.DataFrame(archive_df.name.value_counts())
names.loc[[x.islower() for x in archive_df.name.value_counts().index]]

,name
a,55
the,8
an,7
very,5
quite,4
just,4
one,4
mad,2
getting,2
not,2


It seems clear that _name_ column requires some cleaning
> **Some names are not real pet names**

Also, I'm quite intrigued to know what does the source column represent. Loooking at tweeter's [api documentation](https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object.html). Source gives information as to where data has been originated. Let's have a look at the values that this column takes

In [317]:
archive_df.source.value_counts()

<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>     2221
<a href="http://vine.co" rel="nofollow">Vine - Make a Scene</a>                          91
<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>                       33
<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>      11
Name: source, dtype: int64

It seems that for this dataframe sample, only four values are possible. It can be therefore regarded as categorical data instead of the awful way with which this information is represented. Let's clean this column so categories are clearer when looking at this dataframe
> **Source column contains categories in XML format**

What about the ratings, we know the denominator needs to be always 10, but is this the case?

In [250]:
archive_df.describe()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_status_user_id,rating_numerator,rating_denominator
count,2.356000e+03,7.700000e+01,7.700000e+01,1.810000e+02,1.810000e+02,2356.000000,2356.000000
mean,7.427716e+17,7.440692e+17,2.040329e+16,7.720400e+17,1.241698e+16,13.126486,10.455433
std,6.856705e+16,7.524295e+16,1.260797e+17,6.236928e+16,9.599254e+16,45.876648,6.745237
min,6.660209e+17,6.658147e+17,1.185634e+07,6.661041e+17,7.832140e+05,0.000000,0.000000
25%,6.783989e+17,6.757073e+17,3.589728e+08,7.186315e+17,4.196984e+09,10.000000,10.000000
50%,7.196279e+17,7.032559e+17,4.196984e+09,7.804657e+17,4.196984e+09,11.000000,10.000000
75%,7.993373e+17,8.233264e+17,4.196984e+09,8.203146e+17,4.196984e+09,12.000000,10.000000
max,8.924206e+17,8.862664e+17,8.405479e+17,8.874740e+17,7.874618e+17,1776.000000,170.000000


We can see above that denominator includes at least one vlaue of 170. Also a value of 0. This is wrong as we expect all denominators to be 10. Let's see what is the text for those denominators != 10

In [255]:
archive_df[ archive_df['rating_denominator'] != 10 ].text.values[0:5]

array(["@jonnysun @Lin_Manuel ok jomny I know you're excited but 960/00 isn't a valid rating, 13/10 is tho",
       '@docmisterio account started on 11/15/15',
       'The floofs have been released I repeat the floofs have been released. 84/70 https://t.co/NIYC820tmd',
       'Meet Sam. She smiles 24/7 &amp; secretly aspires to be a reindeer. \nKeep Sam smiling by clicking and sharing this link:\nhttps://t.co/98tB8y7y7t https://t.co/LouL5vdvxx',
       'RT @dog_rates: After so many requests, this is Bretagne. She was the last surviving 9/11 search dog, and our second ever 14/10. RIP https:/…'],
      dtype=object)

We can see a few issues with the denominator that we will need to address in the cleaning stage. Also the numerator seems to showcast similar stats, let's see a few examples

In [258]:
archive_df[ (archive_df['rating_numerator'] > 20) | (archive_df['rating_numerator'] < 10)].text.values[0:5]

array(['This is Bella. She hopes her smile made you smile. If not, she is also offering you her favorite monkey. 13.5/10 https://t.co/qjrljjt948',
       '@dhmontgomery We also gave snoop dogg a 420/10 but I think that predated your research',
       '@s8n You tried very hard to portray this good boy as not so good, but you have ultimately failed. His goodness shines through. 666/10',
       "This is Jerry. He's doing a distinguished tongue slip. Slightly patronizing tbh. You think you're better than us, Jerry? 6/10 hold me back https://t.co/DkOBbwulw1",
       '@markhoppus 182/10'], dtype=object)

The numerator is harder to clean... sometimes it's not clear whether it was a typing error or an intentional high or low score... However instances like the first one that contains decimal numbers are not properly parsed and needs fixing
> **Denominator column contains non 10 values**

> **Numerator has not been properly parsed in some cases**

Also when looking at dog categories, it can be seen that data is **messy**. We find that there are four columns that can be grouped into one as they are referring to the same variable.
> **Archive_df contains messy dog category data**

Let's now have a look at the second dataframe, that is image_df

In [302]:
image_df.head(5)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


It's fair to say that this dataframe shows issues with structure. p1, p2 and p3 refer to the same variable. The same applies to pX_conf and pX_dog where X in [1,2,3]. Therefore this dataframe needs cleaning
> **Same variable spread across a few columns**

Also, why are there columns that indicate whether the prediction is a dog or not. Let's have a look at what happens when some of these are wrong

In [313]:
image_df[ (image_df['p1_dog'] == False) | (image_df['p2_dog'] == False) | (image_df['p3_dog'] == False) ].head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
6,666051853826850816,https://pbs.twimg.com/media/CT5KoJ1WoAAJash.jpg,1,box_turtle,0.933012,False,mud_turtle,0.045885,False,terrapin,0.017885,False
7,666055525042405380,https://pbs.twimg.com/media/CT5N9tpXIAAifs1.jpg,1,chow,0.692517,True,Tibetan_mastiff,0.058279,True,fur_coat,0.054449,False
8,666057090499244032,https://pbs.twimg.com/media/CT5PY90WoAAQGLo.jpg,1,shopping_cart,0.962465,False,shopping_basket,0.014594,False,golden_retriever,0.007959,True
17,666104133288665088,https://pbs.twimg.com/media/CT56LSZWoAAlJj2.jpg,1,hen,0.965932,False,cock,0.033919,False,partridge,0.000052,False
18,666268910803644416,https://pbs.twimg.com/media/CT8QCd1WEAADXws.jpg,1,desktop_computer,0.086502,False,desk,0.085547,False,bookcase,0.079480,False


It seems that some of the images are either not a dog, or contain an object which is not a dog, or the algorithm prediction is wrong. In either case, and since we are dealing with data that relates to dogs, then I think it's cleaner to have data with dogs-only predictions
> **Some predictions are not dogs**

As for the last dataframe tweeter_df it belongs to the same observational unit as archive_df so we should _join_ both of them
> **Twitter_df same observational unit as archive_df**

#### Quality
- Some retweets contain duplicate information
- Some column types are wrong (IDs are floats. Timestamps are strings)
- Some replies contain duplicate information
- Expanded_urls contain missing information
- Some names are not pet names
- Denominator contains non 10 values
- Numerator parsing is not entirely correct
- Some predictions are not dogs
- Source column contains categories in XML format

#### Tidiness
- Dog category is messy
- Image_df same variable uses a few columns
- Twitter_df belongs to the same observational unit as archive_df



<a id='clean'></a>
## Clean

### Missing Data

##### Define

##### Code

Now that the required information has been collected via Tweeter API, then we need to join it with data in archive_df

In [ ]:
archive_df = archive_df.merge(df_tweeter,on='tweet_id',how='left')
archive_df.head(1)

##### Test